In [1]:
import numpy as np

with open('../../1268-0.txt', 'r') as fp:
    text = fp.read()
    
start_index = text.find('THE MYSTERIOUS ISLAND')
end_index = text.find('End of the Project Gutenberg')
text = text[start_index:end_index]
char_set = set(text)
print('Total length: ', len(text))

Total length:  1112350


In [2]:
print('Unique Characters: ', len(char_set))

Unique Characters:  80


In [3]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

text_encoded = np.array([char2int[ch] for ch in text],
                       dtype = np.int32)
print('Text encoded shape: ', text_encoded.shape)

Text encoded shape:  (1112350,)


In [4]:
print(text[:15], '--encoding---> ', text_encoded[:15])

THE MYSTERIOUS  --encoding--->  [44 32 29  1 37 48 43 44 29 42 33 39 45 43  1]


In [5]:
print(text_encoded[15:21], '---reverse---> ', ''.join(char_array[text_encoded[15:21]]))

[33 43 36 25 38 28] ---reverse--->  ISLAND


In [6]:
# create a tensorflow dataset
import tensorflow as tf
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)
# verify
for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

44 -> T
32 -> H
29 -> E
1 ->  
37 -> M


In [7]:
seq_length = 40
chunk_size = seq_length + 1
ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

# function to split x and y
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

In [8]:
# verify
for example in ds_sequences.take(2):
    print('Input, x: ',repr(''.join(char_array[example[0].numpy()])))
    print('Target, y: ',repr(''.join(char_array[example[1].numpy()])))

Input, x:  'THE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced b'
Target, y:  'HE MYSTERIOUS ISLAND ***\n\n\n\n\nProduced by'
Input, x:  ' Anthony Matonak, and Trevor Carlson\n\n\n\n'
Target, y:  'Anthony Matonak, and Trevor Carlson\n\n\n\n\n'


In [9]:
# divide the dataset into mini-batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [10]:
# define an RNN model using Keras Sequential class
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences = True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [11]:
# training parameters
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

In [12]:
# define model
tf.random.set_seed(1)
model = build_model(vocab_size = charset_size,
                   embedding_dim = embedding_dim,
                   rnn_units = rnn_units)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         20480     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1574912   
_________________________________________________________________
dense (Dense)                (None, None, 80)          41040     
Total params: 1,636,432
Trainable params: 1,636,432
Non-trainable params: 0
_________________________________________________________________


In [13]:
# train model
model.compile(optimizer = 'adam',
             loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(ds, epochs = 20)

Epoch 1/20
424/424 [==============================] - 320s 749ms/step - loss: 2.7378
Epoch 2/20
424/424 [==============================] - 330s 775ms/step - loss: 1.8178
Epoch 3/20
424/424 [==============================] - 271s 636ms/step - loss: 1.5882
Epoch 4/20
424/424 [==============================] - 265s 622ms/step - loss: 1.4553
Epoch 5/20
424/424 [==============================] - 247s 580ms/step - loss: 1.3805
Epoch 6/20
424/424 [==============================] - 255s 598ms/step - loss: 1.3213
Epoch 7/20
424/424 [==============================] - 247s 580ms/step - loss: 1.2786
Epoch 8/20
424/424 [==============================] - 284s 669ms/step - loss: 1.2436
Epoch 9/20
424/424 [==============================] - 342s 792ms/step - loss: 1.2206
Epoch 10/20
424/424 [==============================] - 254s 595ms/step - loss: 1.1930
Epoch 11/20
424/424 [==============================] - 219s 516ms/step - loss: 1.1745
Epoch 12/20
424/424 [==============================] - 216s 507

In [17]:
tf.random.set_seed(1)
logits = [[1.0, 1.0, 1.0]]
print('Probabilities: ', tf.math.softmax(logits).numpy()[0])

Probabilities:  [0.33333334 0.33333334 0.33333334]


In [18]:
samples = tf.random.categorical(logits=logits, num_samples=10)
tf.print(samples.numpy())

array([[0, 0, 1, 2, 0, 0, 0, 0, 1, 0]])


In [19]:
def sample(model, starting_str, len_generated_text = 500,
          max_input_length = 40, scale_factor = 1.0):
    
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))
    
    generated_str = starting_str
    model.reset_states()
    
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)
        scaled_logits = logits * scale_factor
        new_char_indx = tf.random.categorical(scaled_logits,
                                             num_samples = 1)
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()
        generated_str += str(char_array[new_char_indx])
        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat([encoded_input, new_char_indx],
                                 axis = 1)
        encoded_input = encoded_input[:, -max_input_length:]
        
    return generated_str

### Try different scaling factor, to test for behavior difference

In [20]:
tf.random.set_seed(1)
print(sample(model, starting_str = 'The Island'))

The Island was obliged to separate
the latter about the palisade. The rooms turn to safe raising
himself in the darkerque reporter, “and I fearing able to remark.”

During the boat?” cried Neb. He suited the boat was proud Pencroft, for everything!”

“What can is not a moment than laster rushed like those six height a hour efflace of extreme shot, why sparkled themose vessel, hisimate of rocks. A narrow point of the settlers were soon torn a few minutes, on the
evergrety of the piles, each other, and the 


In [21]:
logits = np.array([[1.0, 1.0, 3.0]])
print("Probabilities before scaling: ",
      tf.math.softmax(logits).numpy()[0])

Probabilities before scaling:  [0.10650698 0.10650698 0.78698604]


In [22]:
print("Probabilities after 0.5 scaling: ",
      tf.math.softmax(0.5 * logits).numpy()[0])

Probabilities after 0.5 scaling:  [0.21194156 0.21194156 0.57611688]


In [23]:
print("Probabilities after 0.1 scaling: ",
      tf.math.softmax(0.1 * logits).numpy()[0])
# will give near-uniform probabilities

Probabilities after 0.1 scaling:  [0.31042377 0.31042377 0.37915245]


In [24]:
tf.random.set_seed(1)
# scale factor = 2, stable
print(sample(model, starting_str = 'The island',
            scale_factor = 2.0))

The island was so feared that the bridge was not a few days afterwards the interior of the corral, and the colonists were not more than five minutes after the waves had disappeared, and the ball had been seen that she would
have had no doubt that they were all the position the colonists, now accomplished his eyes.

The colonists left the sailor and his companions were therefore put off the corral was not to be feared the door, and the time the cart was not a malignant precaution of the cavern, and the sea


In [25]:
# scale factor = 0.5, more randomness
print(sample(model, starting_str = 'The island',
            scale_factor = 0.5))

The island
veilbated zulreate vi, rowervan, doveled imenyced from
them an
eart,” replied Gideox SUNevary!

Bo’,-of
whom crunkp, though this toh, forewards.-!”0 whole ut, yir withs, an hourbrafat’s return on the vessel an hast?”

Pisig a tinder’ must, caje no carro, ofing.
jumaged only Harding wanting smallow strett!”

Top! Gideon finater-Jainetuatebrin reageedly tates.
Hings.

“Ow5-thers? I
fourt
Jithougn aravi!’s!
colmiNargue ordshight on Toiplos--ir by, with-Alread,? Sidving had rulAlto-zincamescent bir
